## TÍNH TOÁN VÀ VẼ BIỂU ĐỒ TRÒN CỦA TRANG BỊ TỘC VÀ HỆ

In [170]:
import pandas as pd
import numpy as np
import ast
from itertools import chain

### ĐỌC DỮ LIỆU ĐỂ LẤY TẤT CẢ CÁC MÓN ĐỒ

In [171]:
item_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['List Name Item']).drop_duplicates().reset_index(drop=True)

### LẤY RA SỐ LƯỢNG ĐỒ CÓ TRONG TRẬN ĐẤU

ast.literak_eval: dùng để biến đổi string thành list

In [172]:
list_items = []
for i in range(len(item_df['List Name Item'])):
    list_items.append(ast.literal_eval(item_df['List Name Item'][i])) 

### TÌM RA MÓN ĐỒ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

hàm zip dùng để biến 2 list được cho ra thành key và values

lisst(chain) dùng để ép mảng xuống còn 1D và sau đó sort lại

In [173]:
def get_best_worst_items(list_items):
    flatten_list = list(chain.from_iterable(list_items))
    flatten_list = list(chain.from_iterable(flatten_list))
    flatten_list = np.array(flatten_list, dtype = object)
    unique, counts = np.unique(flatten_list, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_item = list(b.items())
    worst_item = list(b.items())[0:5]
    return best_item,worst_item

In [174]:
best_item,worst_item = get_best_worst_items(list_items)

5 món đồ tốt nhất và tệ nhất

In [175]:
print(best_item[:5], worst_item)

[('TFT5_Item_MorellonomiconRadiant', 12), ('TFT5_Item_ShroudOfStillnessRadiant', 12), ('TFT5_Item_FrozenHeartRadiant', 14), ('TFT5_Item_LastWhisperRadiant', 14), ('TFT5_Item_RedemptionRadiant', 14)] [('TFT5_Item_MorellonomiconRadiant', 12), ('TFT5_Item_ShroudOfStillnessRadiant', 12), ('TFT5_Item_FrozenHeartRadiant', 14), ('TFT5_Item_LastWhisperRadiant', 14), ('TFT5_Item_RedemptionRadiant', 14)]


### TÌM VỊ TRÍ TRUNG BÌNH CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU


khi người chơi sử dụng món đồ đó thì sẽ được lọt vào top mấy của game

In [176]:
average_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['Puuid', 'Placement', 'List Name Item']).drop_duplicates().reset_index(drop = True)

Chuyển đổi từ string sang list bởi vì đọc từ csv ra các list sẽ bị chuyển thành string

In [177]:
def convert_string_to_list(ok, placement, column):
    temp = ok[ok.eq(placement).any(1)].reset_index(drop=True)
    top = []
    for i in range(len(temp)):
        top.append(ast.literal_eval(temp[column][i]))
    return top

lọc ra trong những top từ 1 đến 8 số lần item đó được sử dụng

In [178]:
item_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(average_df, i, 'List Name Item')
    temp1 = 'top_' + str(i)
    item_on_top[temp1] = get_best_worst_items(temp)

top những item được sử dụng khi người chơi đạt top 1(thấp đến cao)

In [179]:
best_item_top_1, worst_item_top_1 = item_on_top['top_1']
print(best_item_top_1, "                ", worst_item_top_1)

[('TFT5_Item_RapidFirecannonRadiant', 1), ('TFT5_Item_StatikkShivRadiant', 1), ('TFT5_Item_ArchangelsStaffRadiant', 2), ('TFT5_Item_ChaliceOfPowerRadiant', 2), ('TFT5_Item_GargoyleStoneplateRadiant', 2), ('TFT5_Item_GuinsoosRagebladeRadiant', 2), ('TFT5_Item_HextechGunbladeRadiant', 2), ('TFT5_Item_InfinityEdgeRadiant', 2), ('TFT5_Item_RedemptionRadiant', 2), ('TFT5_Item_BloodthirsterRadiant', 3), ('TFT5_Item_DragonsClawRadiant', 3), ('TFT5_Item_FrozenHeartRadiant', 3), ('TFT5_Item_LastWhisperRadiant', 3), ('TFT5_Item_QuicksilverRadiant', 3), ('TFT5_Item_SpearOfShojinRadiant', 3), ('TFT5_Item_ZzRotPortalRadiant', 3), ('TFT5_Item_BlueBuffRadiant', 4), ('TFT5_Item_DeathbladeRadiant', 4), ('TFT5_Item_HandOfJusticeRadiant', 4), ('TFT5_Item_MorellonomiconRadiant', 4), ('TFT5_Item_RunaansHurricaneRadiant', 4), ('TFT5_Item_ZephyrRadiant', 4), ('TFT5_Item_BrambleVestRadiant', 5), ('TFT5_Item_GuardianAngelRadiant', 5), ('TFT5_Item_LocketOfTheIronSolariRadiant', 5), ('TFT5_Item_RabadonsDeathcapR

### TỈ LỆ CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA MÓN ĐỒ TRONG TỪNG TOP CỦA TRẬN ĐẤU

In [180]:
all_item_df = pd.read_csv("item_tft_csv.csv", usecols=["Item"])

In [181]:
all_items = all_item_df['Item'].to_list()

item_avg_top: vị trí trung bình của món đồ khi ở trong top từ 1 đến 8

item_avg_on_match: tỉ lệ số trận đấu từ top 1 đến 8 mà món đồ đó được sử dụng

In [182]:
sum = 0
temp = 0
item_avg_top = []
item_avg_on_match = []
match_each_top = []
for item in all_items:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(item_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(item_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(item_on_top['top_' + str(i)][0])[item])
        temp2.append(sum/32000)
        item_avg_on_match.append(temp2)
    temp1.append(temp/sum)
    item_avg_top.append(temp1)
    temp = 0
    sum = 0

In [183]:
print(item_avg_top[0], item_avg_on_match[0])

[4.0321428571428575] [0.00153125, 0.00290625, 0.00409375, 0.00509375, 0.0063125, 0.007, 0.00778125, 0.00875]


## TƯƠNG TỰ VỚI TỘC VÀ HỆ

#### đọc dữ liệu để lấy hệ và tộc

In [184]:
traits_df = pd.read_csv('process_data_tft_general_csv.csv', usecols = ['List Traits']).drop_duplicates().reset_index(drop=True)

In [185]:
list_traits = []

for i in range(len(traits_df['List Traits'])):
    list_traits.append(ast.literal_eval(traits_df['List Traits'][i])) 

### TÌM RA TỘC HỆ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

In [186]:
def get_best_worst_traits(list_traits):
    flatten_list = list(chain.from_iterable(list_traits))
    flatten_list = np.array(flatten_list, dtype = object)
    cc = flatten_list[0::3]
    unique, counts= np.unique(cc, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_traits = list(b.items())
    worst_traits = list(b.items())[:5]
    return best_traits, worst_traits

In [187]:
a, b = get_best_worst_traits(list_traits)
print(a,"                ", b)

[('Set8_Arsenal', 1865), ('Set8_Corrupted', 2397), ('Set8_Forecaster', 2778), ('Set8_Supers', 3134), ('Set8_Recon', 3143), ('Set8_Deadeye', 3392), ('Set8_Defender', 3591), ('Set8_Ace', 4028), ('Set8_Renegade', 4058), ('Set8_Civilian', 4618), ('Set8_ExoPrime', 4744), ('Set8_Hacker', 4860), ('Set8_Admin', 5002), ('Set8_Duelist', 5282), ('Set8_AnimaSquad', 5423), ('Set8_Threat', 5858), ('Set8_GenAE', 6030), ('Set8_Heart', 6587), ('Set8_SpaceCorps', 6711), ('Set8_Channeler', 6763), ('Set8_Prankster', 7683), ('Set8_UndergroundThe', 7837), ('Set8_Brawler', 8371), ('Set8_OxForce', 8558), ('Set8_Mascot', 8655), ('Set8_StarGuardian', 9310), ('Set8_Aegis', 10518)]                  [('Set8_Arsenal', 1865), ('Set8_Corrupted', 2397), ('Set8_Forecaster', 2778), ('Set8_Supers', 3134), ('Set8_Recon', 3143)]


### VỊ TRÍ TRUNG BÌNH CỦA TỘC VÀ HỆ

In [188]:
list_traits_df = pd.read_csv('process_data_tft_general_csv.csv', usecols= ['Placement', 'List Traits'])

tộc và hệ tốt nhất và tệ nhất ở top 1

In [189]:
traits_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(list_traits_df, i, 'List Traits')
    temp1 = 'top_' + str(i)
    traits_on_top[temp1] = get_best_worst_traits(temp)

In [190]:
best_traits_on_top_1, worst_traits_on_top_1 = traits_on_top['top_1']
print(best_traits_on_top_1, "              ", worst_traits_on_top_1)

[('Set8_Arsenal', 599), ('Set8_Recon', 690), ('Set8_Deadeye', 876), ('Set8_Corrupted', 908), ('Set8_Ace', 1037), ('Set8_Defender', 1067), ('Set8_Forecaster', 1115), ('Set8_Hacker', 1125), ('Set8_Renegade', 1361), ('Set8_Admin', 1511), ('Set8_Supers', 1592), ('Set8_Duelist', 1643), ('Set8_GenAE', 1727), ('Set8_ExoPrime', 1775), ('Set8_SpaceCorps', 1793), ('Set8_Civilian', 1826), ('Set8_AnimaSquad', 1867), ('Set8_Threat', 1893), ('Set8_Channeler', 1988), ('Set8_Prankster', 2083), ('Set8_UndergroundThe', 2302), ('Set8_OxForce', 2513), ('Set8_Brawler', 2556), ('Set8_Heart', 2565), ('Set8_Mascot', 2765), ('Set8_StarGuardian', 2907), ('Set8_Aegis', 3250)]                [('Set8_Arsenal', 599), ('Set8_Recon', 690), ('Set8_Deadeye', 876), ('Set8_Corrupted', 908), ('Set8_Ace', 1037)]


### TỈ LỆ CỦA TỘC HỆ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA TỘC HỆ TRONG TỪNG TOP CỦA TRẬN ĐẤU

traits_avg_top: tộc hệ trung bình trong các top trận đấu

traits_avg_on_match: tần suất xuất hiện của tộc hệ trong trận đấu

In [191]:
all_traits_df = pd.read_csv('TocHe.csv', usecols=['Trait'])

In [192]:
all_traits = all_traits_df['Trait'].to_list()

traits_avg_top:  vị trí trung bình của tộc hệ khi ở trong top từ 1 đến 8

traits_avg_on_match: tỉ lệ số trận đấu từ top 1 đến 8 mà tộc hệ đó được sử dụng

In [193]:
sum = 0
temp = 0
traits_avg_top = []
traits_avg_on_match = []
for item in all_traits:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(traits_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(traits_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(traits_on_top['top_' + str(i)][0])[item]) 
        temp2.append(sum/32000)
        traits_avg_on_match.append(temp2)
    temp1.append(temp/sum)
    traits_avg_top.append(temp1)
    
    temp = 0
    sum = 0

In [194]:
print(traits_avg_top[0], traits_avg_on_match[0])

[4.304307974335472] [0.062125, 0.1213125, 0.18178125, 0.23821875, 0.2930625, 0.34475, 0.39675, 0.44321875]
